In [1]:
import torch
import random
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Value:
    
 def __init__(self,data,_children=(),_op='',label=''): 
  self.data=data
  self.grad=0.0
  self.backward=lambda:None
  self._prev=set(_children)
  self._op= _op
  self.label=label

 def __repr__(self):
    return f"Value(data={self.data})"
    
 def __add__(self,other):
  out =Value(self.data + other.data,(self,other),'+')
    
  def _backward():
    self.grad=1.0+out.grad
    other.grad=1.0+out.grad
  out._backward=_backward

  return out
    
 def __mul__(self,other):
    
  out =Value(self.data *other.data,(self,other),'*')
    
  def _backward():
    self.grad=other.data*out.grad
    other.grad=self.data*out.grad
  out._backward=_backward

  return out
    
 def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += (other * self.data**(other-1)) * out.grad
    out._backward = _backward

    return out

 def relu(self):
    out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

    def _backward():
        self.grad += (out.data > 0) * out.grad
    out._backward = _backward

    return out
 def tanh(self):
  x=self.data
  t=Value(math.exp(2**x)-1)/(math.exp(2**x)+1)
  out=Value(t,(self,),'tanh')

  def _backward():
    self.grad+=(1-t**2)*out.grad
  out._backward=_backward

  return out

 def exp(self):
  x=self.data
  out=Value(math.exp(x),(self,),'exp')
    
  def _backward():
    self.grad=out.data*out.grad
  out._backward=_backward
    
  return out
    
 def backward(self):
    topo=[]
    visited=set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
    build_topo(self)
    self.grad=1.0
    for node in reversed(topo):
       node._backward()


 def __neg__(self):
    return self * -1

 def __radd__(self, other): 
    return self + other

 def __sub__(self, other): 
    return self + (-other)

 def __rsub__(self, other):
    return other + (-self)

 def __rmul__(self, other):
    return self * other

 def __truediv__(self, other):
    return self * other**-1

 def __rtruediv__(self, other): 
    return other * self**-1    

In [7]:
from graphviz import Digraph

def trace(root):
    nodes,edges=set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes,edges

def draw_dot(root):
    dot =Digraph(format='svg',graph_attr={'rankdir':'LR'})
    nodes,edges=trace(root)
    
    for n in nodes:
        uid=str(id(n))
        dot.node(name=uid,label="{data %.4f}"%(n.data,),shape='record')
        if n._op:
            dot.node(name=uid+n._op,label=n._op)
            dot.edge(uid+n._op,uid)
    for n1,n2 in edges:
        dot.edge(str(id(n1)),str(id(n2))+n2._op)
    return dot


In [8]:
class Neuron:
    def __init__(self,nin):
        self.w=[Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b=Value(random.uniform(-1,1))

    def __call__(self,x):
        print(list(zip(self.w,x)))
        return 0.0

x= [2.0,3.0]
n=Neuron(2)
n(x)
a=Value(2.0)
b=Value(-3.0)
c=Value(10.0)
d=a*b+c



[(Value(data=0.6255934954549751), 2.0), (Value(data=0.10370573573002861), 3.0)]
